In [1]:
import os
import cv2
import numpy as np

In [20]:
sampleDir = r'C:\Users\Mark\Documents\Projects\databank\train\NIR\imgset0594'
imageList = os.listdir(sampleDir)
hrImgDir = os.path.join(sampleDir, imageList[0])
hrImg = cv2.imread(hrImgDir, 0)
hrImg

array([[38, 39, 39, ..., 32, 32, 32],
       [39, 39, 39, ..., 33, 33, 33],
       [39, 39, 39, ..., 34, 34, 34],
       ...,
       [40, 40, 40, ..., 40, 42, 43],
       [40, 40, 40, ..., 40, 41, 42],
       [40, 41, 41, ..., 40, 41, 42]], dtype=uint8)

In [31]:
qMapImgDir = os.path.join(sampleDir, imageList[22])
qMap = cv2.imread(qMapImgDir, 0)
qMap.shape

(128, 128)

In [27]:
qMapImgDir

'C:\\Users\\Mark\\Documents\\Projects\\databank\\train\\NIR\\imgset0594\\QM000.png'